In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import (LinearRegression, HuberRegressor,
                              	RANSACRegressor, TheilSenRegressor)
from sklearn.decomposition import PCA
import palettable
from pathlib import Path
import glob
import os
import scipy.ndimage as ndimage
from scipy import signal
import statsmodels.api as smapi
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

### Linear Regression for Indivudual Gesture

In [2]:
#Specify your data path here
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit']

#Specity the use of filter
low_filt = True
g_filt = False

for move in movements:
    print(f'For {move}:')
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    X = tcs.iloc[:,1:]
    y = tcs.iloc[:,0]
    
    if low_filt:
        sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
        X_f = signal.sosfilt(sos, X)
    elif g_filt:
        #print('After Gaussian filter:')
        X = np.array(X)
        X_f = ndimage.gaussian_filter(X,(0,3))
    else:
        X_f = X

    reg = LinearRegression().fit(X_f, y)    
    y_pred = reg.predict(X_f)


    mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = reg.score(X_f, y)

    print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    print(f'The R2 Score of {move} for all candidates: {r2} \n')
    
    # Monitor the data quality of individual candidate of a specific gesture
    if move == 'Leg_extension_sit':
        for csv in csv_filenames:
            print(csv)
            t = pd.read_csv(csv,index_col = False, header = None)
            t = t.loc[:, :5]
            #print(tcs.iloc[:5,:])
            X_indi = t.iloc[:,1:]
            y_indi = t.iloc[:,0]
            #model = 
            y_pred_indi = reg.predict(X_indi)
            mse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=True)
            rmse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=False)
            mae_indi = mean_absolute_error(y_true=y_indi, y_pred=y_pred_indi)
            r2_indi = reg.score(X_indi, y_indi)
            print(f'The Root Mean Squared Error (RMSE) of {move} for {csv} candidates: {rmse_indi}')
            print(f'The Mean Absolute Error (MAE) of {move} for {csv} candidates: {mae_indi}')
            print(f'The R2 Score of {move} for {csv} candidates: {r2_indi} \n')

For HC_90:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 10.072696803362135
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.138736890168994
The R2 Score of HC_90 for all candidates: 0.8773157502694977 

For HC_45:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 7.47491448628599
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.73625740212967
The R2 Score of HC_45 for all candidates: 0.8261202895907329 

For Leg_presses_liftup:
The Root Mean Squared Error (RMSE) of Leg_presses_liftup for all candidates: 9.103975470733586
The Mean Absolute Error (MAE) of Leg_presses_liftup for all candidates: 6.302031362760883
The R2 Score of Leg_presses_liftup for all candidates: 0.9073990135866516 

For Half_squat:
The Root Mean Squared Error (RMSE) of Half_squat for all candidates: 7.8885487457643215
The Mean Absolute Error (MAE) of Half_squat for all candidates: 5.126824580842276
The R2 Score of Half_squat for all candidates: 0.867714181711885

### Linear Regression for All Gestures

In [11]:
#linear regression for all movements
all_filenames = []
for path, subdirs, files in os.walk(data_dir):
    for name in files:
        #print(os.path.join(path, name))
        all_filenames.append(os.path.join(path, name))

tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in all_filenames))
tcs = tcs.loc[:, :5]
low_filt = True
g_filt = True

X = tcs.iloc[:,1:]
y = tcs.iloc[:,0]

if low_filt:
    #tcs = np.array(tcs)
    #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
    #tcs_filt = pd.DataFrame(tcs_filt)
    sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
    X = signal.sosfilt(sos, X)
if g_filt:
    X = np.array(X)
    X = ndimage.gaussian_filter(X,(0,3))

reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)


mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
mae = mean_absolute_error(y_true=y, y_pred=y_pred)
r2 = reg.score(X, y)

#print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
print(f'The Root Mean Squared Error (RMSE) of all movements for all candidates: {rmse}')
print(f'The Mean Absolute Error (MAE) of all movements for all candidates: {mae}')
print(f'The R2 Score of all movements for all candidates: {r2} \n')

The Root Mean Squared Error (RMSE) of all movements for all candidates: 11.338720513901864
The Mean Absolute Error (MAE) of all movements for all candidates: 7.745907596714453
The R2 Score of all movements for all candidates: 0.8595552950690997 



### Try splitting the timeseries to back & forth

In [4]:
#calculate derivative of angles
dx = 0.01
de = np.gradient(y, dx)

In [5]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit']

low_filt = True
g_filt = False
for move in movements:
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    
    #calculate the derivative of the angles and split back and forth by comparing the derivative with 0
    #de<0 means back and de>0 means forth
    dx = 0.01
    de = np.gradient(tcs.iloc[:,0], dx)
    tcs['deri'] = de
    back = tcs[tcs.deri < 0].iloc[:,:5]
    forth = tcs[tcs.deri > 0].iloc[:,:5]
    
    
    for p in ['back','forth']:
        print(f'For {move} {p}:')
        if p == 'back':
            tc = back
        if p == 'forth':
            tc = forth
    
        X = tc.iloc[:,1:]
        y = tc.iloc[:,0]

        if low_filt:
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X_f = signal.sosfilt(sos, X)
        if g_filt:
            #print('After Gaussian filter:')
            X = np.array(X)
            X_f = ndimage.gaussian_filter(X,(0,3))


        
        reg = LinearRegression().fit(X_f, y)
        y_pred = reg.predict(X_f)

        
        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For HC_90 back:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 11.116002009490993
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.9894027253124715
The R2 Score of HC_90 for all candidates: 0.8565020632528504 

For HC_90 forth:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 9.130583721246115
The Mean Absolute Error (MAE) of HC_90 for all candidates: 5.790788561873758
The R2 Score of HC_90 for all candidates: 0.8939778662894731 

For HC_45 back:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 8.263460854349617
The Mean Absolute Error (MAE) of HC_45 for all candidates: 5.230064021870939
The R2 Score of HC_45 for all candidates: 0.7886731315224049 

For HC_45 forth:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 6.968560181589126
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.530360543505167
The R2 Score of HC_45 for all candidates: 0.8477558329675827 

For Leg_extension_sit back:
The Root Mean Sq

# Linear Regression of individual sensors

In [6]:
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit']

low_filt = True
g_filt = False
for move in movements:
    print(f'For {move}:')
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]

    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    print('For Individual Sensor Linear Regression:\n')
    y = tcs.iloc[:,0]
    
    for i in range(5):
        print(f'Sensor {i+1}:')
        X = tcs.iloc[:,i+1]

        if low_filt:
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X = np.array(X).reshape(-1,1)
            X_f = signal.sosfilt(sos, X)
        elif g_filt:
            #print('After Gaussian filter:')
            X = np.array(X).reshape(-1,1)
            X_f = ndimage.gaussian_filter(X,(0,3))
        else:
            X_f = np.array(X)
            
            X_f = X_f.reshape(-1,1)

        reg = LinearRegression().fit(X_f, y)

        y_pred = reg.predict(X_f)


        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For HC_90:
For Individual Sensor Linear Regression:

Sensor 1:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 18.01721054335499
The Mean Absolute Error (MAE) of HC_90 for all candidates: 12.035276187736645
The R2 Score of HC_90 for all candidates: 0.6074704146617103 

Sensor 2:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 11.315739828780938
The Mean Absolute Error (MAE) of HC_90 for all candidates: 7.390337193738411
The R2 Score of HC_90 for all candidates: 0.8451671187567696 

Sensor 3:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 10.656476043398103
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.791862811711212
The R2 Score of HC_90 for all candidates: 0.8626829343910499 

Sensor 4:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 13.64179422255173
The Mean Absolute Error (MAE) of HC_90 for all candidates: 9.037911699824567
The R2 Score of HC_90 for all candidates: 0.7749700743405912 

Sensor 5:
The Ro

# only using sensor 2,3 & 4

In [7]:
print('For Individual movement of all candidates:')
#Specify your data path here
data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

#movements = ['HC_90','HC_45','Leg_extension_sit','Leg_presses_liftup', 'Half_squat']
movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit']

#Specity the use of filter
low_filt = True
g_filt = False

for move in movements:
    print(f'For {move}:')
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    X = tcs.iloc[:,2:-1]
    y = tcs.iloc[:,0]
    print(X.shape)
    if low_filt:
        sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
        X_f = signal.sosfilt(sos, X)
    elif g_filt:
        #print('After Gaussian filter:')
        X = np.array(X)
        X_f = ndimage.gaussian_filter(X,(0,3))
    else:
        X_f = X

    reg = LinearRegression().fit(X_f, y)    
    y_pred = reg.predict(X_f)


    mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
    rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
    mae = mean_absolute_error(y_true=y, y_pred=y_pred)
    r2 = reg.score(X_f, y)

    print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
    print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
    print(f'The R2 Score of {move} for all candidates: {r2} \n')
    
    # Monitor the data quality of individual candidate of a specific gesture
    if move == 'Leg_extension_sit':
        for csv in csv_filenames:
            print(csv)
            t = pd.read_csv(csv,index_col = False, header = None)
            t = t.loc[:, :5]
            #print(tcs.iloc[:5,:])
            X_indi = t.iloc[:,1:]
            y_indi = t.iloc[:,0]
            #model = 
            y_pred_indi = reg.predict(X_indi)
            mse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=True)
            rmse_indi = mean_squared_error(y_true=y_indi, y_pred=y_pred_indi, squared=False)
            mae_indi = mean_absolute_error(y_true=y_indi, y_pred=y_pred_indi)
            r2_indi = reg.score(X_indi, y_indi)
            print(f'The Root Mean Squared Error (RMSE) of {move} for {csv} candidates: {rmse_indi}')
            print(f'The Mean Absolute Error (MAE) of {move} for {csv} candidates: {mae_indi}')
            print(f'The R2 Score of {move} for {csv} candidates: {r2_indi} \n')

For Individual movement of all candidates:
For HC_90:
(456091, 3)
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 10.49352419141085
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.798552481156599
The R2 Score of HC_90 for all candidates: 0.8668503514537745 

For HC_45:
(484153, 3)
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 7.790424496958704
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.957769279389106
The R2 Score of HC_45 for all candidates: 0.8111318629395876 

For Leg_presses_liftup:
(452249, 3)
The Root Mean Squared Error (RMSE) of Leg_presses_liftup for all candidates: 10.126648444170383
The Mean Absolute Error (MAE) of Leg_presses_liftup for all candidates: 7.06938780069803
The R2 Score of Leg_presses_liftup for all candidates: 0.885426304358696 

For Half_squat:
(453795, 3)
The Root Mean Squared Error (RMSE) of Half_squat for all candidates: 8.556883809322237
The Mean Absolute Error (MAE) of Half_squat for all cand

In [13]:
#linear regression for all movements
print('For all movements of all candidates:')
all_filenames = []
for path, subdirs, files in os.walk(data_dir):
    for name in files:
        #print(os.path.join(path, name))
        all_filenames.append(os.path.join(path, name))

tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in all_filenames))
tcs = tcs.loc[:, :5]
low_filt = True
g_filt = True

X = tcs.iloc[:,2:-1]
y = tcs.iloc[:,0]

if low_filt:
    #tcs = np.array(tcs)
    #tcs_filt = ndimage.gaussian_filter(tcs,(0,3))
    #tcs_filt = pd.DataFrame(tcs_filt)
    sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
    X = signal.sosfilt(sos, X)
if g_filt:
    X = np.array(X)
    X = ndimage.gaussian_filter(X,(0,3))

reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)


mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
mae = mean_absolute_error(y_true=y, y_pred=y_pred)
r2 = reg.score(X, y)

#print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
print(f'The Root Mean Squared Error (RMSE) of all movements for all candidates: {rmse}')
print(f'The Mean Absolute Error (MAE) of all movements for all candidates: {mae}')
print(f'The R2 Score of all movements for all candidates: {r2} \n')

For all movements of all candidates:
The Root Mean Squared Error (RMSE) of all movements for all candidates: 12.867852991997813
The Mean Absolute Error (MAE) of all movements for all candidates: 8.518868334932073
The R2 Score of all movements for all candidates: 0.8191204577722054 



In [12]:
#calculate derivative of angles
print('For back&forth of individual movement of all candidates:')
dx = 0.01
de = np.gradient(y, dx)

data_dir = Path(r'C:\Users\Firefly\OneDrive - Imperial College London\Data for Yilin\Formal_data_collection')

movements = ['HC_90','HC_45','Leg_presses_liftup', 'Half_squat','sit']

low_filt = True
g_filt = False
for move in movements:
    #csv_filenames = []
    os.chdir(Path(data_dir, move))
    extension = 'csv'
    csv_filenames = [i for i in glob.glob('*.{}'.format(extension))]
    
    tcs = pd.concat((pd.read_csv(f,index_col = False, header = None) for f in csv_filenames))
    tcs = tcs.loc[:, :5]
    
    #calculate the derivative of the angles and split back and forth by comparing the derivative with 0
    #de<0 means back and de>0 means forth
    dx = 0.01
    de = np.gradient(tcs.iloc[:,0], dx)
    tcs['deri'] = de
    back = tcs[tcs.deri < 0].iloc[:,:5]
    forth = tcs[tcs.deri > 0].iloc[:,:5]
    
    
    for p in ['back','forth']:
        print(f'For {move} {p}:')
        if p == 'back':
            tc = back
        if p == 'forth':
            tc = forth
    
        X = tc.iloc[:,2:-1]
        y = tc.iloc[:,0]

        if low_filt:
            sos = signal.butter(1, 1.5, 'low', fs=100, output='sos')
            X_f = signal.sosfilt(sos, X)
        if g_filt:
            #print('After Gaussian filter:')
            X = np.array(X)
            X_f = ndimage.gaussian_filter(X,(0,3))


        
        reg = LinearRegression().fit(X_f, y)
        y_pred = reg.predict(X_f)

        
        mse = mean_squared_error(y_true=y, y_pred=y_pred, squared=True)
        rmse = mean_squared_error(y_true=y, y_pred=y_pred, squared=False)
        mae = mean_absolute_error(y_true=y, y_pred=y_pred)
        r2 = reg.score(X_f, y)

        #print(f'The Mean Squared Error of HC90 for all candidates: {mse}')
        print(f'The Root Mean Squared Error (RMSE) of {move} for all candidates: {rmse}')
        print(f'The Mean Absolute Error (MAE) of {move} for all candidates: {mae}')
        print(f'The R2 Score of {move} for all candidates: {r2} \n')


For back&forth of individual movement of all candidates:
For HC_90 back:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 11.32788535242671
The Mean Absolute Error (MAE) of HC_90 for all candidates: 7.3267732900798865
The R2 Score of HC_90 for all candidates: 0.8509794666650315 

For HC_90 forth:
The Root Mean Squared Error (RMSE) of HC_90 for all candidates: 9.32188332196653
The Mean Absolute Error (MAE) of HC_90 for all candidates: 6.08434996380336
The R2 Score of HC_90 for all candidates: 0.8894886766070885 

For HC_45 back:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 8.35721862264279
The Mean Absolute Error (MAE) of HC_45 for all candidates: 5.336190012642468
The R2 Score of HC_45 for all candidates: 0.783850469745106 

For HC_45 forth:
The Root Mean Squared Error (RMSE) of HC_45 for all candidates: 7.0160176352752615
The Mean Absolute Error (MAE) of HC_45 for all candidates: 4.600886178053715
The R2 Score of HC_45 for all candidates: 0.8456751383

# classification